In [1]:
import plotly.tools as tls
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import pandas_ta as ta
import pandas as pd
import plotly.express as px
from datetime import date
from datetime import timedelta

In [2]:
# securities to be included in the portfolio building process
securities_list = ['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT']
returnSeriesList = []
# fetch data from yahoo finance and calculate the daily percent return using pandas_ta lib
for s in securities_list:
    returnSeriesList.append(pdr.DataReader(s, data_source='yahoo', start=str(date.today() - timedelta(days=730)),
                                           end=str(date.today() - timedelta(days=1))).ta.percent_return(length=1))
returns_df = pd.concat(returnSeriesList, axis=1)
returns_df.columns = securities_list
returns_df.dropna(inplace=True)

In [3]:
# Risk-free rate taken as US Treasury Bill rate
risk_free_rate = 0.0151
num_assets = len(securities_list)
# generate 10000 portfolios and choose best one
num_portfolios = 10000

In [4]:
# get mean returns and covariance
mean_returns = returns_df.mean()
cov_returns = returns_df.cov()

In [ ]:
# plot graph to view the volatility of the stocks
returns_graph = plt.figure()
plt.plot(returns_df.to_numpy(), alpha=.4)
plt.xlabel("Days")
plt.ylabel("Returns")
plt.legend(securities_list)
returns_graph = tls.mpl_to_plotly(returns_graph)
returns_graph.layout.height = 400
returns_graph.layout.width = 800
returns_graph.show()
returns_graph.write_image("stockReturns.png")

In [6]:
# generate random weight vector for making portfolio
def getWeightVector(n):
    weights = np.random.random(n)
    return weights / sum(weights)

In [7]:
# calculate the returns and volatility of the portfolio
def portfolioPerformance(mean_returns, cov_returns, weight_vector, num_trading_days):
    ret = np.sum(mean_returns * weight_vector) * num_trading_days
    weight_vec_trans = weight_vector.T
    std = np.sqrt(np.dot(weight_vec_trans, np.dot(cov_returns, weight_vector))) * np.sqrt(252)
    return std, ret

In [8]:
# generate the specified number of portfolios and calculate their performance
def generatePortfolios():
    portfolio_weights = []
    portfolio_results = np.zeros((3, num_portfolios))
    for i in range(num_portfolios):
        weight_vector = getWeightVector(num_assets)
        portfolio_weights.append(weight_vector)
        portfolio_results[0, i], portfolio_results[1, i] = portfolioPerformance(mean_returns, cov_returns,
                                                                                weight_vector, 252)
        # calculate the Sharpe Ratio
        portfolio_results[2, i] = (portfolio_results[1, i] - risk_free_rate) / portfolio_results[0, i]
    return portfolio_results, portfolio_weights

In [ ]:
portfolio_results, portfolio_weights = generatePortfolios()
# get index position of portfolio with highest Sharpe Ratio
best_sharpe_ratio_index = np.argmax(portfolio_results[2])
best_portfolio_volatility, best_portfolio_annl_return = portfolio_results[0, best_sharpe_ratio_index],portfolio_results[1, best_sharpe_ratio_index]
# plot the risk and returns of all portfolios and highlight the chosen portfolio (highest sharpe ratio) in red
portfolios_graph = plt.figure()
plt.plot(portfolio_results[0, :], portfolio_results[1, :], 'o', markersize=5)
plt.plot(best_portfolio_volatility, best_portfolio_annl_return, 'r-o', markersize=15)
plt.xlabel('Risk')
plt.ylabel('Returns')
portfolios_graph = tls.mpl_to_plotly(portfolios_graph)
portfolios_graph.layout.height = 400
portfolios_graph.layout.width = 800
portfolios_graph.show()
portfolios_graph.write_image("portfolios.png")

In [ ]:
# View the composition of the optimal portfolio along with its performance
best_portfolio_alloc = pd.DataFrame(portfolio_weights[best_sharpe_ratio_index], index=securities_list,
                                    columns=['Allocation'])
best_portfolio_alloc['Allocation'] = [round(i * 100, 2) for i in best_portfolio_alloc['Allocation']]
best_portfolio_alloc = best_portfolio_alloc.T
print("Best portfolio details")
print("Annualised Returns {:.2f}%".format(best_portfolio_annl_return * 100))
print("Annualised Volatility {:.2f}%".format(best_portfolio_volatility * 100))
print("Securities distribution: ")
print(best_portfolio_alloc)

In [ ]:
# Display pie chart as visual aid for portfolio composition
fig = px.pie(best_portfolio_alloc.T, values='Allocation', names=best_portfolio_alloc.T.index, title='Optimal Portfolio')
fig.show()
fig.write_image("optimalPortfolioComposition.png")